In [ ]:
#| default_exp regularize.regularize_callback

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
from fastai.callback.all import *
from fastcore.basics import store_attr, listify
from fasterai.core.criteria import *
from fasterai.core.granularity import *
from fasterai.core.schedule import *

import torch
import torch.nn as nn
from typing import Union, Optional, Type

## Overview

The `RegularizeCallback` applies structured regularization during training to encourage weight sparsity at various granularities. This is useful as a pre-pruning step: by regularizing groups of weights toward zero during training, subsequent pruning can remove more parameters with less accuracy loss.

**Key Features:**
- Supports multiple granularity levels (`'weight'`, `'vector'`, `'kernel'`, `'filter'`)
- Compatible with any criteria from `fasterai.core.criteria`
- Optional scheduling to vary regularization strength over training

In [ ]:
#| export
class RegularizeCallback(Callback):
    def __init__(self, 
                 criteria: Union[Criteria, list[Criteria]],            # Criteria(s) to use for regularization
                 granularity: Union[str, list[str]],                   # Granularity level(s) for grouping
                 weight: float = 0.01,                                 # Regularization weight
                 layer_types: Union[Type, list[Type]] = nn.Conv2d,     # Layer types to apply regularization to
                 schedule: Optional[Schedule] = None,                  # Optional schedule for regularization weight
                 per_layer_weights: Optional[dict[str, float]] = None, # Optional per-layer weights
                 verbose: bool = False                                 # Whether to report regularization weight
    ):
        "Callback to apply regularization using criteria during training"
        store_attr()
        self.criteria = listify(criteria)
        self.granularity = listify(granularity)
        self.layer_types = listify(layer_types)
        self.per_layer_weights = per_layer_weights or {}
        self.current_weight = weight
        
    def before_batch(self) -> None:
        "Update regularization weight if scheduled"
        if self.schedule is not None:
            self.current_weight = self.schedule([self.weight], self.pct_train)[0]
        
    def after_loss(self) -> None:
        "Apply regularization after computing the main loss"
        reg = self.get_norm()
        self.learn.loss_grad += reg
        self.learn.loss = self.learn.loss_grad.clone()
    
    def _iter_layers(self):
        "Iterate over matching layers with weights"
        for m in self.learn.model.modules():
            if any(isinstance(m, lt) for lt in self.layer_types) and hasattr(m, 'weight'):
                yield m
            
    def get_norm(self) -> torch.Tensor:
        "Compute regularization using the specified criteria and granularities"
        # Pre-filter modules once
        layers = list(self._iter_layers())
        
        layer_regs = []
        for crit in self.criteria:
            for g in self.granularity:
                for m in layers:
                    try:
                        scores = crit.f(m.weight)[None].abs().sum(Granularities.get_dim(m, g))
                        layer_regs.append(self.current_weight * scores.sum())
                    except (KeyError, ValueError) as e:
                        import warnings
                        warnings.warn(f"Skipping regularization for {type(m).__name__}: {e}")
                    except RuntimeError as e:
                        import warnings
                        warnings.warn(f"Runtime error in regularization for {type(m).__name__}: {e}")
        
        return torch.stack(layer_regs).sum() if layer_regs else torch.tensor(0.0)
    
    def after_epoch(self) -> None:
        "Report current regularization weight if verbose"
        if self.verbose:
            print(f"Current regularization weight: {self.current_weight:.6f}")

In [ ]:
show_doc(RegularizeCallback)

**Parameters:**
- `criteria`: Importance criteria to use for computing regularization (e.g., `large_final`)
- `granularity`: Level at which to group weights (`'weight'`, `'vector'`, `'kernel'`, `'filter'`)
- `weight`: Regularization coefficient (higher = stronger regularization)
- `layer_types`: Module types to regularize (default: `nn.Conv2d`)
- `schedule`: Optional schedule to vary regularization strength over training
- `verbose`: Print regularization weight after each epoch

---

## Usage Example

Apply filter-level L1 regularization to encourage entire filters to become unimportant (making them easier to prune later):

In [ ]:
# from fasterai.regularize.regularize_callback import RegularizeCallback
# from fasterai.core.criteria import large_final

# # Apply L1 regularization at filter granularity
# cb = RegularizeCallback(
#     criteria=large_final,
#     granularity='filter',
#     weight=0.01,
#     verbose=True
# )

# learn.fit(10, cbs=[cb])

**Typical Workflow:**
1. Train with `RegularizeCallback` to push unimportant filter groups toward zero
2. After training, use `PruneCallback` or `Pruner` to remove the zeroed-out structures
3. Fine-tune the pruned model to recover any lost accuracy

---

## See Also

- [Sparsifier](../sparse/sparsifier.html) - Apply sparsification after regularization pushes weights to zero
- [Criteria](../core/criteria.html) - Importance measures that can leverage regularized weights
- [SparsifyCallback](../sparse/sparsify_callback.html) - Combine with sparsification for gradual pruning